# 4. Types for agents

We are getting towards our goal of putting everything together to make a model of people moving around and interacting with one another. Most people start off susceptible, but when an infectious person meets a susceptible the infection is transmitted with a certain probability

We will make an **individual-based model**, also called an **agent-based model**. We need a `struct` called `Agent` that contains whatever information an agent needs. In our case we will need a position and an infection status.

The position will behave almost like a normal random walk that we have seen before, while the infection status needs to reflect whether the agent is susceptible (S), infectious (I) or recovered / removed (R).

## Enums

We could represent the infection status simply using an integer, e.g. 0, 1 or 2. 
But then our code will be hard to read, since we will be comparing the infection status to numbers all the time without remembering which one is which.

Rather we would prefer to have the symbols `S`, `I` and `R`. 
We could just define global constants for these, but there is a better alternative: an `Enum`.

`Enum` is short for "*enum*erated type". We want a new *type*, `InfectionStatus`, with possible values `S`, `I` and `R`. 
    We can achieve this as follows:

In [12]:
@enum InfectionStatus S=1 I R   # specify that `S` corresponds to the value 1 

In [13]:
InfectionStatus

Enum InfectionStatus:
S = 1
I = 2
R = 3

We see that I is an object of type `InfectionStatus`, which has the value 2, in the sense that we can convert it to an integer:

In [3]:
status = I
Int(status)

2

In [4]:
status

I::InfectionStatus = 2

We can check the status with

In [5]:
if status == I
    println("infected!")
end

infected!


In [6]:
if status == 3
     println("recovered!")
end

In [8]:
status == I

true

In [9]:
Int(status) == 2

2

In [14]:
S < I

true

In [15]:
I < S

false

In our application the integer values corresponding to each state are arbitrary, but in other applications you may want to give meaning to the numerical values.

## Agent type: Inheritance

How can we create an `Agent` type for an agent that lives in 2D? Perhaps the simplest way is as follows:

In [16]:
struct SimpleAgent1
    x::Int64
    y::Int64
    status::InfectionStatus
end

[Note that in Julia you cannot redefine a `struct` to have different fields in the same session. We will make several versions so we are just adding a number to each one.]

#### Exercise

1. Create an object of type `SimpleAgent1`.


2. Write a method of `move` for `SimpleAgent1`.

In [17]:
InfectionStatus(2)

I::InfectionStatus = 2

In [18]:
InfectionStatus(5)

ArgumentError: ArgumentError: invalid value for Enum InfectionStatus: 5

In [19]:
methods(SimpleAgent1)

# 2 methods for type constructor:
[1] SimpleAgent1(x::Int64, y::Int64, status::InfectionStatus) in Main at In[16]:2
[2] SimpleAgent1(x, y, status) in Main at In[16]:2

In [25]:
a = SimpleAgent1(1, 2, S)

SimpleAgent1(1, 2, S)

In [21]:
SimpleAgent1(1, 2, 1)

MethodError: MethodError: Cannot `convert` an object of type Int64 to an object of type InfectionStatus
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at essentials.jl:171
  InfectionStatus(::Integer) at Enums.jl:186

In [22]:
SimpleAgent1(1, 2, InfectionStatus(1))

SimpleAgent1(1, 2, S)

In [23]:
move(a::SimpleAgent1) = SimpleAgent1(a.x + 1, a.y, a.status)

move (generic function with 1 method)

In [24]:
infect(a::SimpleAgent1) = SimpleAgent1(a.x, a.y, I)

infect (generic function with 1 method)

In [26]:
a = SimpleAgent1(1, 2, S)

SimpleAgent1(1, 2, S)

In [28]:
infect(a)

SimpleAgent1(1, 2, I)

In [29]:
a

SimpleAgent1(1, 2, S)

In [30]:
a = infect(a)

SimpleAgent1(1, 2, I)

The problem with this is that we have duplicated codfe: `move` for `SimpleAgent1` looks identical to the one for `Walker2D`. 

One way to reduce duplication is using a form of **inheritance**, by making both types **subtypes** of a common type. The subtype relation is written `X <: Y` (read "`X` is a subtype of `Y`"). The common type is an **abstract type**:

In [32]:
abstract type AbstractWalker2D end

In [ ]:
struct SimpleWalker2D <: AbstractWalker2D
    x::Int64
    y::Int64
end

In [ ]:
struct SimpleAgent2 <: AbstractWalker2D
    x::Int64
    y::Int64
    status::InfectionStatus
end

Since these two types share common fields, we can write the `move` method just *once*, giving the abstract type as the type annotation.

#### Exercise

1. How can you create an object of type `SimpleAgent2`?


2. Can you extend the `move` function such that it works for both `SimpleWalker2D` and `SimpleAgent2`?

## Composition

An alternative method, which [is usually more powerful and general](https://en.wikipedia.org/wiki/Composition_over_inheritance), is to use **composition**, i.e. placing one object *inside* another:

In [9]:
struct SimpleWalker2D
    x::Int64
    y::Int64
end

In [11]:
@enum InfectionStatus S I R

In [12]:
struct Agent1
    position::SimpleWalker2D
    status::InfectionStatus
end

At first glance this may look a little strange, since we seem to be saying that an agent contains a walker. But really it is just a way to maintain the data inside its pacakge and be able to reuse the infrastructure that we have already built up for that type.

#### Exercise

1. How can you construct an object of type `Agent1`?


2. How would you extend `move` for `Agent1`?

In [13]:
move(w::SimpleWalker2D) = SimpleWalker2D(w.x + 1, w.y)

move (generic function with 1 method)

In [14]:
w = SimpleWalker2D(1, 2)

(1, 2)

In [15]:
a = Agent1(w, S)

Agent1((1, 2), S)

In [16]:
Agent1(SimpleWalker2D(1, 2), S)

Agent1((1, 2), S)

Extend `Base.show`:

In [17]:
function Base.show(io::IO, w::SimpleWalker2D)
    print(io, "($(w.x), $(w.y))")
end

In [18]:
function Base.show(io::IO, a::Agent1)
    print(io, "Agent object with position $(a.position) and status $(a.status)")
end

In [19]:
a

Agent object with position (1, 2) and status S

To change only one field inside an object: SetField.jl, Lens.jl

## Parametrised types

However, now suppose we want to have agents that live in 1D or 2D, or on a network etc. Currently we would have to make different types `Agent2D`, `Agent1D`, ... that look almost identical.

We have already seen a few times that whenever we observe **code duplication** like this, it's a hint that there is a possible **abstraction**. 
Basically we want to be able to say "an agent has some kind of position, and an infection status".

We could write this as follows:

In [20]:
struct Agent2
    position
    status::InfectionStatus
end

In [21]:
Agent2(3, S)

Agent2(3, S)

In [22]:
Agent2("hello", S)

Agent2("hello", S)

with no type annotation on `position`. However, it turns out that this is *very bad for performance* in Julia.

> Never define a composite type with untyped or abstractly-typed fields. 

[Unless you really know what you are doing, or don't care at all about performance.]

To get high performance we need to always allow Julia to be able to work out the types of everything.

The correct solution is to use a **parametrised type**:

In [ ]:
struct Agent3{P}
    position::P
    status::InfectionStatus
end

In [23]:
struct Agent3_Int64
    position::Int64
end

In [24]:
struct Agent3_Float64
    position::Float64
end

In [27]:
struct Agent4{T}   # "allow any type T"
    position::T
end

In [28]:
Agent4(1)

Agent4{Int64}(1)

In [29]:
Agent4("hello:")

Agent4{String}("hello:")

Here, `P` is a **type parameter**. You can think of it as a special kind of variable that holds a type. Julia will work out what this type should be, based on the kind of object you supply for the `position` field.

*If* there is a common abstract type `AbstractWalker` for all of the possible types that we want to be able to use for `P`, then we can instead write

In [ ]:
struct Agent4{P <: AbstractWalker}
    position::P
    status::InfectionStatus
end

This *restricts* the allowed types that can be used for `P`, which is often desirable. However, it may exclude other possible types, for example `Int`, which cannot be made into a subtype of `AbstractWalker`. 
In this case the previous, unrestricted version may be better.

[You can also use `Union` to specify different allowed types.]

In [30]:
struct Agent5{P}
    position::P
    status::InfectionStatus
end

In [31]:
Agent5(3, S)

Agent5{Int64}(3, S)

In [32]:
Agent5("hello", S)

Agent5{String}("hello", S)

In [33]:
Agent5(Agent5(3, S), S)

Agent5{Agent5{Int64}}(Agent5{Int64}(3, S), S)

#### Exercise

1. Construct objects of type `Agent3` with different types for `P`. Hint: You *don't need* to explicitly tell Julia the type parameter &ndash; it should be worked out automatically!

Exercise: Write a `move` function

In [34]:
w = SimpleWalker2D(1, 2)

a = Agent5(w, S)

Agent5{SimpleWalker2D}((1, 2), S)

In [35]:
Agent5

Agent5

In [37]:
move(a::Agent5) = Agent5(move(a.position), a.status)

move (generic function with 2 methods)

In [38]:
a

Agent5{SimpleWalker2D}((1, 2), S)

In [39]:
move(a)

Agent5{SimpleWalker2D}((2, 2), S)

## Walkers in any number of dimensions (N)

In [40]:
struct SimpleWalker{N,T}
   position::NTuple{N,T} 
end

In [41]:
SimpleWalker( (1, 2) )

SimpleWalker{2,Int64}((1, 2))

In [42]:
SimpleWalker( (1, 2, -1, 10, 17) )

SimpleWalker{5,Int64}((1, 2, -1, 10, 17))

In [43]:
using StaticArrays

In [44]:
struct SimpleWalker2{N,T}
   position::SVector{N,T} 
end

In [46]:
w = SimpleWalker2(SA[-1, 2, 5.0])

SimpleWalker2{3,Float64}([-1.0, 2.0, 5.0])

In [47]:
w.position

3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 -1.0
  2.0
  5.0

In [48]:
w.position + SVector(1, 2, 3)

3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 0.0
 4.0
 8.0

In [49]:
p = w.position

3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 -1.0
  2.0
  5.0

In [51]:
setindex(p, 10, 2)

3-element SArray{Tuple{3},Float64,1,3} with indices SOneTo(3):
 -1.0
 10.0
  5.0

In [52]:
setindex( (1, 2), 10, 2)

(1, 10)